In [1]:
!pip install langchain_community unstructured

  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 3.1 MB/s eta 0:00:003.0 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached cryptography-46.0.3-cp311-abi3-manylinux_2_34_x86_64.whl.metadata (5.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.5 MB/s eta 0:00:00 MB/s eta 0:00:01:01
Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 7.9 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 8.1 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.8/167.8 kB 7

In [1]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

In [18]:
loader = UnstructuredMarkdownLoader(
    "./docs/llms-full.txt",
    mode = 'elements', # single, elements
    strategy = 'fast'
)

In [19]:
docs = loader.load()

In [20]:
# for i in range(10,20):
#     print(docs[i].page_content)
len(docs)

6044

In [21]:
docs[10]

Document(metadata={'source': './docs/llms-full.txt', 'link_texts': ['Auth', 'resource-level access control'], 'link_urls': ['/langsmith/langgraph-python-sdk#langgraph_sdk.auth.Auth', '/langsmith/auth#single-owner-resources'], 'languages': ['eng'], 'file_directory': './docs', 'filename': 'llms-full.txt', 'filetype': 'text/markdown', 'last_modified': '2025-10-21T14:44:53', 'parent_id': 'ec02236cd0ab191e4257202b0d334b21', 'category': 'NarrativeText', 'element_id': 'c83d6114f113e6e0d8a283ebd05204d9'}, page_content="You'll keep the same Auth object and resource-level access control, but upgrade authentication to use Supabase as your identity provider. While Supabase is used in this tutorial, the concepts apply to any OAuth2 provider. You'll learn how to:")

In [28]:

import networkx as nx

G = nx.DiGraph()

element_map = {doc.metadata.get('element_id'): doc for doc in docs}
for doc in docs:
    node_id = doc.metadata.get('element_id')
    G.add_node(node_id, content=doc.page_content, **doc.metadata)

    # Parent relationship
    parent_id = doc.metadata.get('parent_id')
    if parent_id:
        G.add_edge(parent_id, node_id, relation='parent')

    # Hyperlinks (resolve if possible)
    for link in doc.metadata.get('link_urls', []):
        # If link maps to another element_id, create edge
        if link in element_map:
            G.add_edge(node_id, link, relation='hyperlink')

print(f"Graph built with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")



Graph built with 6044 nodes and 5252 edges.


In [36]:
node_ids = [id for id in G.nodes]

In [46]:
[G.nodes[node_ids[i]]['content'] for i in range(0,10)]

['null',
 'Source: https://docs.langchain.com/index',
 'Documentation',
 "LangChain is the platform for agent engineering. AI teams at Replit, Clay, Rippling, Cloudflare, Workday, and more trust LangChain's products to engineer reliable agents. Our **open source frameworks** help you build agents: * [**LangChain**](/oss/python/langchain/overview) helps you quickly get started building agents, with any model provider of your choice. * [**LangGraph**](/oss/python/langgraph/overview) allows you to control every step of your custom agent with low-level orchestration, memory, and human-in-the-loop support. You can manage long-running tasks with durable execution. [**LangSmith**](/langsmith/home) is a platform that helps AI teams use live production data for continuous testing and improvement. LangSmith provides: * **Observability** to see exactly how your agent thinks and acts with detailed tracing and aggregate trend metrics. * **Evaluation** to test and score agent behavior on production 

In [48]:
edge_id_tuples = [each_edge for each_edge in G.edges()]

In [49]:
len(edge_id_tuples)

5252

In [66]:
edge_dict = defaultdict(list)
for i in range(0,10):
    parent, child = edge_id_tuples[i]
    edge_dict[parent].append(child)

In [78]:
items = []
n_items = 0
for key in edge_dict.keys():
    n_items += len(edge_dict[key])
    for item in edge_dict[key]:
        items.append(item)
    

In [81]:
n_items, items, edge_dict

(10,
 ['a8d45a9c8a4a76033fc38190eb4a6ab5',
  'f50a7a06166f0af92ded1c54425d7f0a',
  '843ef2bbf170f886f46cefd6d48eefef',
  'f21c908fd3c8dd684094cc261547ae9e',
  '5eefd73f87dd20f398f5651d6205eadf',
  '331a5069c7e3d23929834ea5afc3087e',
  '3d3f109f636bf00e6301ff0e85ac3de0',
  'c83d6114f113e6e0d8a283ebd05204d9',
  'a9121089bc580813c00565fa2b820e5c',
  '960d6719a4eb78dba596b11af912394c'],
 defaultdict(list,
             {'81c1937ca9f54ff4d90a0d9369495dc7': ['a8d45a9c8a4a76033fc38190eb4a6ab5'],
              '96f3c4c50d8d095d91f50a2b99380295': ['f50a7a06166f0af92ded1c54425d7f0a',
               '843ef2bbf170f886f46cefd6d48eefef',
               'f21c908fd3c8dd684094cc261547ae9e',
               '5eefd73f87dd20f398f5651d6205eadf'],
              'ec02236cd0ab191e4257202b0d334b21': ['331a5069c7e3d23929834ea5afc3087e',
               '3d3f109f636bf00e6301ff0e85ac3de0',
               'c83d6114f113e6e0d8a283ebd05204d9',
               'a9121089bc580813c00565fa2b820e5c',
               '960d6719a4

In [89]:
for key in edge_dict['96f3c4c50d8d095d91f50a2b99380295']:
    print(key, G.nodes[key]['content'], "\n")

f50a7a06166f0af92ded1c54425d7f0a LangChain is the platform for agent engineering. AI teams at Replit, Clay, Rippling, Cloudflare, Workday, and more trust LangChain's products to engineer reliable agents. Our **open source frameworks** help you build agents: * [**LangChain**](/oss/python/langchain/overview) helps you quickly get started building agents, with any model provider of your choice. * [**LangGraph**](/oss/python/langgraph/overview) allows you to control every step of your custom agent with low-level orchestration, memory, and human-in-the-loop support. You can manage long-running tasks with durable execution. [**LangSmith**](/langsmith/home) is a platform that helps AI teams use live production data for continuous testing and improvement. LangSmith provides: * **Observability** to see exactly how your agent thinks and acts with detailed tracing and aggregate trend metrics. * **Evaluation** to test and score agent behavior on production data and offline datasets for continuous 

In [60]:
from collections import defaultdict

my_dict = defaultdict(list)
my_dict["even_numbers"].append(2)
my_dict["even_numbers"].append(4)
my_dict["odd_numbers"].append(1)
my_dict["odd_numbers"].append(3)
print(my_dict)

defaultdict(<class 'list'>, {'even_numbers': [2, 4], 'odd_numbers': [1, 3]})


In [65]:
my_dict['odd_numbers']

[1, 3]

In [93]:
i = 0
for doc in docs:
    print(i, len(doc.page_content))
    i+=1

0 4
1 40
2 13
3 1089
4 11
5 28
6 9
7 34
8 60
9 249
10 242
11 40
12 62
13 84
14 10
15 33
16 126
17 121
18 80
19 49
20 128
21 306
22 1030
23 63
24 79
25 201
26 29
27 109
28 156
29 119
30 74
31 60
32 46
33 129
34 13
35 51
36 99
37 222
38 644
39 34
40 58
41 235
42 1869
43 160
44 263
45 439
46 16
47 152
48 25
49 101
50 29
51 231
52 39
53 152
54 179
55 367
56 59
57 35
58 40
59 121
60 10
61 137
62 57
63 59
64 58
65 79
66 80
67 54
68 93
69 113
70 75
71 34
72 66
73 108
74 18
75 1021
76 1069
77 692
78 82
79 177
80 38
81 142
82 144
83 35
84 86
85 112
86 112
87 90
88 3200
89 17
90 106
91 97
92 163
93 45537
94 25
95 100
96 134
97 81
98 258
99 76
100 373
101 750
102 148
103 34
104 129
105 25
106 235
107 48
108 564
109 19
110 51
111 61
112 8
113 195
114 46
115 38
116 78
117 100
118 20
119 32
120 210
121 26
122 13
123 64
124 384
125 252
126 13
127 31
128 47
129 49
130 49
131 53
132 97
133 78
134 29
135 132
136 325
137 13
138 38
139 55
140 21
141 21
142 90
143 14
144 51
145 67
146 58
147 52
148 59
149 

In [94]:
docs[93]

Document(metadata={'source': './docs/llms-full.txt', 'languages': ['eng'], 'file_directory': './docs', 'filename': 'llms-full.txt', 'filetype': 'text/markdown', 'last_modified': '2025-10-21T14:44:53', 'parent_id': '871cd76f3511f0bfaab2c47be55c7893', 'category': 'NarrativeText', 'element_id': '27a28420c6ed54c2148499097c062f7f'}, page_content='1. The breakpoints are set during `compile` time.\n2. `interrupt_before` specifies the nodes where execution should pause before the node is executed.\n3. `interrupt_after` specifies the nodes where execution should pause after the node is executed.\n\nAlternatively, you can set static interrupts at run time:\n\n<Tabs>\n  <Tab title="Python">\n    ```python {highlight={1,5,6}} theme={null}\n    await client.runs.wait( # (1)!\n        thread_id,\n        assistant_id,\n        inputs=inputs,\n        interrupt_before=["node_a"], # (2)!\n        interrupt_after=["node_b", "node_c"] # (3)!\n    )\n    ```\n\n    1. `client.runs.wait` is called with th